# Heuristics - CVRP

[OR-Tools Documentation](https://or-tools.github.io/docs/pdoc/ortools/constraint_solver/pywrapcp.html)

To do:

- Implement skills (and a couple more job types)
- Allow incomplete jobs (to mimic more jobs than time available!)
- Maximise engineer utilisation, currently the method tries to minimise job+drive time!

In [1]:
from itertools import cycle

import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import matplotlib as mpl
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import functions as f
import importlib

## Instantiate data

In [2]:
# assume indx 0 is the dummy end location
dataset = pd.read_csv("./data/appointments.csv", index_col=0)
engineers = pd.read_csv("./data/engineers.csv", index_col=0)

engineers = engineers.loc[engineers.index[:3]]

n_vehicles = len(engineers)
seniority_multiplier = engineers["seniority"].values.tolist()
start_locations = engineers["start"].values.tolist()
end_locations = engineers["end"].values.tolist()
start_times = engineers["start_min"].values.tolist()  # in minutes from 8am
end_times = engineers["end_min"].values.tolist()  # in minutes from 8am


for home_indx in start_locations:
    dataset.loc[home_indx, "timeslot"] = "AD"
    dataset.loc[home_indx, "job_type"] = "start"

coordinates = dataset.loc[:, ["x", "y"]]

N = coordinates.shape[0]

In [3]:
# time windows, each each number represents 1 minute
time_window_dict = {
    "AD": [0, max(end_times)],  # 08:00 - 16:30
    "AM": [0, 240],  # 08:00 - 12:00
    "PM": [240, 510],  # 12:00 - 16:30
}

service_time_dict = {"start": 0, "DF-M Commision": 50, "DF-M Exchange": 100, "end": 0}

In [4]:
time_matrix_multiplier = 1

# assume travel time is proportional to euclidean distance
time_matrix = squareform(pdist(coordinates, metric="euclidean"))

# reduce the drive times to make feasible solutions more likely, will adjust later!
time_matrix = np.round(time_matrix * time_matrix_multiplier, decimals=0).astype(int)

time_matrix[:, 0] = time_matrix[0, :] = (
    0  # the end location is a dummy variable used to allow arbitrary end locations
)

## Model

In [5]:
# Create the routing index manager: number of nodes, number of vehicles, depot node
manager = pywrapcp.RoutingIndexManager(N, n_vehicles, start_locations, end_locations)

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

### Parameters

In [6]:
# must create a different time_callback for each vehicle
def create_time_callback(vehicle_id):
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        travel_time = time_matrix[from_node, to_node]

        # convention is that job time added to drive FROM the job's node
        service_time = int(
            service_time_dict[dataset.loc[from_node, "job_type"]]
            * seniority_multiplier[vehicle_id]
        )

        return travel_time + service_time

    return time_callback


transit_callback_indices = []

# add callbacks so the solver can compute the time between nodes (including service times)
for vehicle_id in range(n_vehicles):
    transit_callback = create_time_callback(vehicle_id)
    transit_callback_index = routing.RegisterTransitCallback(transit_callback)
    transit_callback_indices.append(transit_callback_index)

### Constraints

In [7]:
# Define time window contraints
time = "Time"

# create a CumulVar, for each location AND each vehicle
# 0 to N_locations-1 (index and node order is different!), N_locations to N_locations+N_vehicles-1
# VERY IMPORTANT, the use index for cumVars, not the node numbers
routing.AddDimensionWithVehicleTransits(
    transit_callback_indices,
    max(end_times),  # allow waiting time
    max(
        end_times
    ),  # maximum time per vehicle (and location but the range is set later)
    True,  # start cumul to zero
    time,
)

time_dimension = routing.GetDimensionOrDie(time)

# add time window constraints for each location via the index!
# cannot do for the depot as is special case
for location_indx in dataset.index:
    if location_indx == 0:
        continue
    elif location_indx in start_locations:
        continue

    timeslot = dataset.loc[location_indx, "timeslot"]

    index = manager.NodeToIndex(location_indx)

    time_dimension.CumulVar(index).SetRange(
        time_window_dict[timeslot][0],
        time_window_dict[timeslot][1],  # window depends on promised timeslot
    )

In [8]:
# Add max drive time for each vehicle
# do this by setting the range of the end node for each vehicle
for vehicle_id in range(n_vehicles):

    end_index = routing.End(vehicle_id)
    time_dimension.CumulVar(end_index).SetRange(
        start_times[vehicle_id], end_times[vehicle_id]
    )  # 8.5 hours max

    # ensures engineer start during work hours, not strictly necessary but worth keeping in
    start_index = routing.Start(vehicle_id)
    time_dimension.CumulVar(start_index).SetRange(
        start_times[vehicle_id], end_times[vehicle_id]
    )  # 8.5 hours max

In [9]:
# this for loop encourage the optimizer to minimse the time of the last job of each vehicle
for i in range(n_vehicles):

    # start as early as possible
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.Start(i)))

    # finish as early as possible
    routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

Add penalties for missing each appointment. The [penalty is added to the cost function](https://developers.google.com/optimization/routing/penalties), so should be larger than the max(drive+job) time. Remmeber, the objective funtion minimises total drive+job time. This will encourage the solver to never drop any appointments unless it has to.

In future:

- Appointments that are far away or long will always be dropped first, so might need to be prioritised with smaller penalties.
- Jeap appointments could be given a higher penalty so that they are dropped before already booked in appointments (if desired)

In [10]:
# calculate the penalty
max_possible_drive_time = max(time_matrix.flatten())
longest_possible_job = max(service_time_dict.values()) * max(seniority_multiplier)
penalty = int(max_possible_drive_time + longest_possible_job)

# add penalty to the nodes to allow them to be dropped
for location_indx in dataset.index:
    if location_indx == 0:
        continue
    elif location_indx in start_locations:
        continue

    routing.AddDisjunction([manager.NodeToIndex(location_indx)], penalty)

### Objective

In [11]:
# The cost of each arc is vehicle dependant!
# Must link to correct callback
for vehicle_id in range(n_vehicles):
    routing.SetArcCostEvaluatorOfVehicle(
        transit_callback_indices[vehicle_id], vehicle_id
    )

### Solution

In [12]:
# Setting heuristic strategies
search_parameters = pywrapcp.DefaultRoutingSearchParameters()

search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)

search_parameters.time_limit.FromSeconds(2)

# Solve the problem
solution = routing.SolveWithParameters(search_parameters)

In [13]:
solution.ObjectiveValue()

2858

### Plot Results

In [14]:
importlib.reload(f)
f.print_solution(
    routing,
    n_vehicles,
    start_times,
    end_times,
    time_dimension,
    solution,
    manager,
    time_matrix,
    service_time_dict,
    dataset,
)

Key: |Node| A:Arrival, JT:Job Time, T:Travel Time, J:Job Time, F:Finish Time, WT: Work Time

Vehicle 0: Available 0min -> 510min
|4| A:0, TS: AD, JT:36, T:36, J:0 -> |12| A:36, TS: AM, JT:215, T:55, J:100 -> |13| A:251, TS: PM, JT:155, T:75, J:50 -> |10| A:406, TS: AD, JT:80, T:0, J:50 -> F:486 min
Totals: J:200,    T:166, JT:486, F:486 min
Percentages: J:39%,    T:32%, JT:95%, F:%95%

Vehicle 1: Available 0min -> 400min
|5| A:0, TS: AD, JT:43, T:43, J:0 -> |16| A:43, TS: AD, JT:139, T:89, J:50 -> |3| A:182, TS: AM, JT:75, T:25, J:50 -> |2| A:257, TS: PM, JT:86, T:36, J:50 -> |11| A:343, TS: AD, JT:50, T:0, J:50 -> F:393 min
Totals: J:200,    T:193, JT:393, F:393 min
Percentages: J:50%,    T:48%, JT:98%, F:%98%

Vehicle 2: Available 0min -> 510min
|6| A:0, TS: AD, JT:47, T:47, J:0 -> |18| A:47, TS: AM, JT:159, T:39, J:100 -> |7| A:206, TS: AD, JT:128, T:8, J:100 -> |8| A:334, TS: AD, JT:69, T:9, J:50 -> |1| A:403, TS: PM, JT:60, T:0, J:50 -> F:463 min
Totals: J:300,    T:103, JT:463, F

In [15]:
importlib.reload(f)
dropped_nodes = f.print_appointments(
    dataset,
    time_dimension,
    manager,
    solution,
    start_locations,
    time_window_dict,
    routing,
)

Appointments:
Key: |Node| (timeslot): Att: Arrival Time
AM [0, 240] min, PM [240, 510] min

|1| (PM): Att 403
|2| (PM): Att 257
|3| (AM): Att 182
|7| (AD): Att 206
|8| (AD): Att 334
|9| (AM): Not Attended
|10| (AD): Att 406
|11| (AD): Att 343
|12| (AM): Att 36
|13| (PM): Att 251
|14| (AM): Not Attended
|15| (AM): Not Attended
|16| (AD): Att 43
|17| (AD): Not Attended
|18| (AM): Att 47


In [17]:
importlib.reload(f)
tours = f.compile_tours(n_vehicles, routing, manager, solution, True)
fig = f.plot_tours(tours, coordinates, dropped_nodes)
fig.show()